# Python Fortgeschritten: Environment-Variablen und sichere Konfiguration
## Tag 5 - Notebook 32
***
In diesem Notebook wird behandelt:
- Environment-Variablen mit os
- .env Dateien und python-dotenv
- Sichere Passwort-Verwaltung
- Datenbankbeispiel mit zwei Prozessen
- Best Practices für sensible Daten
***


## 1 Environment-Variablen Grundlagen

**Environment-Variablen** sind Variablen, die außerhalb des Python-Codes gesetzt werden und von der Anwendung zur Laufzeit gelesen werden können. Sie sind ideal für:
- **Konfiguration**: Datenbank-URLs, API-Keys, Port-Nummern
- **Sensible Daten**: Passwörter, Secrets, Tokens
- **Umgebungs-spezifische Einstellungen**: Development, Staging, Production

### Warum Environment-Variablen?

- **Sicherheit**: Sensible Daten werden nicht im Code gespeichert
- **Flexibilität**: Verschiedene Umgebungen ohne Code-Änderungen
- **Best Practice**: Industriestandard für Konfiguration
- **Git-Sicherheit**: Keine sensiblen Daten im Repository

### Wann verwenden?

Environment-Variablen sollten verwendet werden für:
- **Passwörter und Secrets**: Niemals im Code hardcodieren!
- **API-Keys**: Externe Service-Keys
- **Datenbankverbindungen**: URLs, Benutzernamen, Passwörter
- **Umgebungs-Konfiguration**: Development vs. Production
- **Feature-Flags**: Funktionen ein/ausschalten


## 2 .env Dateien und python-dotenv

**`.env` Dateien** sind eine praktische Methode, um Environment-Variablen lokal zu speichern. Das Paket `python-dotenv` lädt diese Dateien automatisch.

### Vorteile von .env Dateien

- **Lokale Entwicklung**: Einfache Konfiguration ohne System-Environment
- **Team-Sharing**: `.env.example` als Template für das Team
- **Git-Sicherheit**: `.env` wird in `.gitignore` aufgenommen
- **Verschiedene Umgebungen**: `.env.development`, `.env.production`

### Best Practice Struktur

```
projekt/
  .env                    # Lokale Konfiguration (NICHT in Git!)
  .env.example            # Template mit Platzhaltern (IN Git)
  .gitignore              # Enthält .env
  config.py               # Lädt .env
```


In [ ]:
# Beispiel: .env Datei erstellen (für Demo)
# In der Praxis würde man diese manuell erstellen

env_content = """# Datenbank-Konfiguration
DB_HOST=localhost
DB_PORT=5432
DB_NAME=mydatabase
DB_USER=admin
DB_PASSWORD=super_secret_password_123

# API-Konfiguration
API_KEY=sk-1234567890abcdef
API_URL=https://api.example.com

# Umgebung
ENVIRONMENT=development
DEBUG=True
"""

# .env Datei schreiben (nur für Demo)
with open('.env', 'w') as f:
    f.write(env_content)

print(".env Datei erstellt!")
print("\nInhalt:")
print(env_content)


In [ ]:
%pip install dotenv

In [ ]:
# python-dotenv installieren (falls nicht vorhanden)
# !pip install python-dotenv

from dotenv import load_dotenv

# .env Datei laden
load_dotenv()  # Lädt automatisch .env aus dem aktuellen Verzeichnis

# Jetzt können wir die Variablen lesen
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')  # Sicher aus .env geladen!

print(f"Datenbank-Host: {db_host}")
print(f"Datenbank-Port: {db_port}")
print(f"Datenbank-Name: {db_name}")
print(f"Datenbank-User: {db_user}")
print(f"Datenbank-Passwort: {'*' * len(db_password) if db_password else 'NICHT GESETZT'}")


## 3 .gitignore für .env Dateien

**WICHTIG**: `.env` Dateien dürfen **NIEMALS** ins Git-Repository! Sie enthalten sensible Daten.

### Best Practice

1. **`.env`** → In `.gitignore` aufnehmen
2. **`.env.example`** → Template-Datei **OHNE** echte Passwörter (darf ins Git)
3. **README.md** → Dokumentation, wie `.env` erstellt wird


In [ ]:
# Beispiel: .env.example erstellen (darf ins Git)
env_example_content = """# Datenbank-Konfiguration
# Kopiere diese Datei zu .env und fülle die Werte aus
DB_HOST=localhost
DB_PORT=5432
DB_NAME=your_database_name
DB_USER=your_username
DB_PASSWORD=your_password_here

# API-Konfiguration
API_KEY=your_api_key_here
API_URL=https://api.example.com

# Umgebung
ENVIRONMENT=development
DEBUG=True
"""

with open('.env.example', 'w') as f:
    f.write(env_example_content)

print(".env.example erstellt (Template für das Team)")
print("\n.gitignore sollte enthalten:")
print("  .env")
print("  .env.local")
print("  .env.*.local")


## 4 Datenbankbeispiel: Zwei Prozesse

In diesem Beispiel zeigen wir:
1. **Prozess 1**: Startet eine Datenbank und wartet auf Verbindungen
2. **Prozess 2**: Verbindet sich zur Datenbank mit sensiblen Daten (Passwort)
3. **Sichere Konfiguration**: Passwort wird aus Environment-Variablen geladen

### Architektur

```
Prozess 1 (DB-Server)          Prozess 2 (Client)
     |                              |
     |-- Startet DB ----------------|
     |                              |
     |<-- Verbindung mit Passwort --|
     |   (aus .env)                 |
     |                              |
     |-- Authentifizierung ---------|
     |                              |
     |<-- Query --------------------|
```


In [ ]:
import sqlite3
import multiprocessing
import time
import os
from pathlib import Path
from dotenv import load_dotenv

# .env laden
load_dotenv()

# Datenbank-Pfad aus Environment-Variable
DB_PATH = os.getenv('DB_PATH', 'example.db')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'default_password')

def setup_database():
    """Prozess 1: Datenbank initialisieren und starten"""
    print(f"[DB-Server] Starte Datenbank-Setup...")
    
    # Alte DB löschen (für Demo)
    if Path(DB_PATH).exists():
        Path(DB_PATH).unlink()
    
    # Datenbank erstellen
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    # Tabelle erstellen
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL,
            email TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Testdaten einfügen
    cursor.execute('''
        INSERT INTO users (username, email) VALUES
        ('alice', 'alice@example.com'),
        ('bob', 'bob@example.com'),
        ('charlie', 'charlie@example.com')
    ''')
    
    conn.commit()
    conn.close()
    
    print(f"[DB-Server] Datenbank erstellt: {DB_PATH}")
    print(f"[DB-Server] Passwort für Authentifizierung: {'*' * len(DB_PASSWORD)}")
    print(f"[DB-Server] Warte auf Verbindungen...")
    
    # Simuliere, dass DB-Server läuft
    time.sleep(2)  # Warte auf Client-Verbindung
    
    print(f"[DB-Server] Datenbank bereit!")
    return DB_PATH


## 5.1 Fortgeschritten: Passwort nur bei Bedarf auslesen (Lazy Loading)

### Warum ist das sinnvoll?

Bei der `DatabaseConfig`-Klasse oben wird das Passwort **einmal beim Start** geladen und dann **permanent im Speicher** gehalten. Das kann ein Sicherheitsrisiko sein.

### Wovor schützt Lazy Loading?

| Bedrohung | Beschreibung | Schutz durch Lazy Loading |
|-----------|--------------|---------------------------|
| **Memory Dumps** | Bei einem Absturz wird der Speicher in eine Datei geschrieben (Core Dump). Passwörter könnten dort im Klartext stehen. | Passwort ist nur kurzzeitig im Speicher |
| **Debugging-Tools** | Ein Angreifer mit Zugriff auf den Prozess kann den Speicher auslesen (z.B. mit `gdb`, Process Explorer). | Zeitfenster für Angriff minimiert |
| **Lange laufende Prozesse** | Server-Anwendungen laufen oft tagelang. Je länger das Passwort im Speicher liegt, desto größer das Risiko. | Passwort nur bei tatsächlicher Verwendung |
| **Logging/Tracing** | Versehentliches Loggen von Objekten könnte das Passwort offenlegen. | Property kann kontrolliert werden |

### Wann ist Lazy Loading besonders wichtig?

- **Server-Anwendungen**: Lange Laufzeiten, viele Verbindungen
- **Shared Hosting**: Mehrere Benutzer auf einem System
- **Compliance-Anforderungen**: PCI-DSS, HIPAA, DSGVO
- **Hochsensible Daten**: Finanz-, Gesundheits-, Personaldaten

### Einschränkung in Python

**Wichtig**: Python-Strings sind *immutable* (unveränderlich). Selbst nach `del password` kann der String noch im Speicher liegen, bis der Garbage Collector ihn entfernt. Für maximale Sicherheit bei hochsensiblen Anwendungen sollten externe Lösungen wie **HashiCorp Vault** oder der **OS Credential Manager** (`keyring`) verwendet werden.


In [ ]:
# Beispiel: Sichere Konfigurations-Klasse mit Lazy Loading

class SecureDatabaseConfig:
    """
    Datenbank-Konfiguration mit Lazy Loading des Passworts.
    
    Das Passwort wird NICHT im Objekt gespeichert, sondern nur
    bei Bedarf aus der Environment-Variable gelesen.
    """
    
    def __init__(self):
        load_dotenv()
        self.host = os.getenv('DB_HOST', 'localhost')
        self.port = int(os.getenv('DB_PORT', '5432'))
        self.name = os.getenv('DB_NAME', 'mydatabase')
        self.user = os.getenv('DB_USER', 'admin')
        # WICHTIG: Passwort wird hier NICHT gespeichert!
    
    @property
    def password(self):
        """
        Liest das Passwort nur wenn es benötigt wird.
        Wird bei jedem Zugriff neu aus der Environment-Variable geladen.
        """
        pwd = os.getenv('DB_PASSWORD')
        if not pwd:
            raise ValueError("DB_PASSWORD nicht gesetzt!")
        return pwd
    
    def connect(self):
        """
        Simuliert eine Datenbankverbindung.
        Das Passwort ist nur kurz im lokalen Scope dieser Funktion.
        """
        print(f"[SecureConfig] Verbinde zu {self.host}:{self.port}/{self.name}")
        
        # Passwort nur hier kurz im Speicher
        pwd = self.password
        print(f"[SecureConfig] Passwort geladen: {'*' * len(pwd)}")
        
        # Hier würde die echte Verbindung hergestellt
        # connection = create_real_connection(self.host, self.user, pwd)
        
        # Nach Verwendung explizit löschen (Best Effort)
        del pwd
        
        print("[SecureConfig] Verbindung hergestellt, Passwort aus lokalem Scope entfernt")
        return True

# Vergleich: Normale vs. Sichere Konfiguration
print("=== Vergleich: Normale vs. Sichere Konfiguration ===\n")

# Normale Konfiguration (Passwort permanent im Speicher)
print("1. Normale DatabaseConfig:")
print("   - Passwort wird in __init__ geladen")
print("   - Bleibt als self.password im Objekt")
print("   - Liegt permanent im Speicher\n")

# Sichere Konfiguration (Passwort nur bei Bedarf)
print("2. SecureDatabaseConfig mit Lazy Loading:")
print("   - Kein self.password Attribut")
print("   - @property liest bei jedem Zugriff neu")
print("   - Passwort nur kurzzeitig im Speicher\n")

# Demo
print("=== Demo: SecureDatabaseConfig ===\n")
secure_config = SecureDatabaseConfig()

# Das Objekt hat kein gespeichertes Passwort
print(f"Gespeicherte Attribute: host={secure_config.host}, user={secure_config.user}")
print(f"Hat 'password' als Instanzvariable? {'_password' in secure_config.__dict__}")

# Erst beim Zugriff wird das Passwort geladen
print("\nVerbindung herstellen (Passwort wird jetzt geladen):")
secure_config.connect()


In [ ]:
def connect_to_database():
    """Prozess 2: Verbindet sich zur Datenbank mit sensiblen Daten"""
    print(f"[Client] Warte auf Datenbank...")
    time.sleep(1)  # Kurz warten, damit DB bereit ist
    
    # Passwort aus Environment-Variable laden (SICHER!)
    # NICHT hardcodiert im Code!
    password = os.getenv('DB_PASSWORD')
    
    if not password:
        print("[Client] FEHLER: DB_PASSWORD nicht gesetzt!")
        print("[Client] Bitte .env Datei erstellen oder Environment-Variable setzen.")
        return
    
    print(f"[Client] Verbinde zur Datenbank...")
    print(f"[Client] Passwort geladen: {'*' * len(password)} (aus .env)")
    
    # Simuliere Authentifizierung
    if password == os.getenv('DB_PASSWORD', ''):
        print(f"[Client] Authentifizierung erfolgreich!")
    else:
        print(f"[Client] Authentifizierung fehlgeschlagen!")
        return
    
    # Verbindung zur Datenbank
    db_path = os.getenv('DB_PATH', 'example.db')
    
    if not Path(db_path).exists():
        print(f"[Client] FEHLER: Datenbank {db_path} nicht gefunden!")
        return
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Query ausführen
    print(f"[Client] Führe Query aus...")
    cursor.execute('SELECT * FROM users')
    users = cursor.fetchall()
    
    print(f"[Client] Gefundene Benutzer:")
    for user in users:
        print(f"  - ID: {user[0]}, Username: {user[1]}, Email: {user[2]}")
    
    conn.close()
    print(f"[Client] Verbindung geschlossen.")


In [ ]:
# Zwei Prozesse starten
print("=== Datenbankbeispiel: Zwei Prozesse ===\n")

# Prozess 1: Datenbank-Server
db_process = multiprocessing.Process(target=setup_database)

# Prozess 2: Client
client_process = multiprocessing.Process(target=connect_to_database)

# Beide Prozesse starten
db_process.start()
client_process.start()

# Warten bis beide fertig sind
db_process.join()
client_process.join()

print("\n=== Beide Prozesse abgeschlossen ===")


## 5 Best Practices: Sichere Passwort-Verwaltung

### NIEMALS so machen:

```python
# SCHLECHT: Passwort hardcodiert im Code
password = "super_secret_password_123"  # NIEMALS!
db.connect(user="admin", password=password)
```

**Warum schlecht?**
- Passwort ist im Code sichtbar
- Wird ins Git-Repository committed
- Jeder mit Code-Zugriff sieht das Passwort
- Kann nicht einfach geändert werden

### RICHTIG: Environment-Variablen verwenden

```python
# GUT: Passwort aus Environment-Variable
import os
from dotenv import load_dotenv

load_dotenv()  # Lädt .env Datei
password = os.getenv('DB_PASSWORD')  # SICHER!
db.connect(user="admin", password=password)
```

**Warum gut?**
- Passwort ist nicht im Code
- `.env` Datei ist in `.gitignore`
- Jeder Entwickler hat seine eigene `.env`
- Einfach zu ändern ohne Code-Änderung


In [ ]:
# Beispiel: Sichere Konfigurations-Klasse

class DatabaseConfig:
    """Sichere Datenbank-Konfiguration mit Environment-Variablen"""
    
    def __init__(self):
        # Lade .env Datei
        load_dotenv()
        
        # Lade Konfiguration aus Environment-Variablen
        self.host = os.getenv('DB_HOST', 'localhost')
        self.port = int(os.getenv('DB_PORT', '5432'))
        self.name = os.getenv('DB_NAME', 'mydatabase')
        self.user = os.getenv('DB_USER', 'admin')
        self.password = os.getenv('DB_PASSWORD')  # Kein Default!
        
        # Validiere, dass Passwort gesetzt ist
        if not self.password:
            raise ValueError(
                "DB_PASSWORD nicht gesetzt! "
                "Bitte .env Datei erstellen oder Environment-Variable setzen."
            )
    
    def get_connection_string(self):
        """Erstellt Connection-String (ohne Passwort zu loggen!)"""
        return f"postgresql://{self.user}:***@{self.host}:{self.port}/{self.name}"
    
    def __repr__(self):
        """String-Repräsentation (ohne Passwort!)"""
        return (
            f"DatabaseConfig("
            f"host={self.host}, "
            f"port={self.port}, "
            f"name={self.name}, "
            f"user={self.user}, "
            f"password=***)"  # Passwort niemals anzeigen!
        )

# Verwendung
try:
    config = DatabaseConfig()
    print("Konfiguration geladen:")
    print(config)
    print(f"\nConnection-String: {config.get_connection_string()}")
    print(f"\nPasswort ist gesetzt: {'Ja' if config.password else 'Nein'}")
except ValueError as e:
    print(f"FEHLER: {e}")


## 6 Checkliste: Sichere Konfiguration

### Was zu tun ist:

1. **`.env` Datei erstellen** mit allen sensiblen Daten
2. **`.env` in `.gitignore` aufnehmen** (NIEMALS committen!)
3. **`.env.example` erstellen** als Template (darf ins Git)
4. **`python-dotenv` verwenden** zum Laden der `.env` Datei
5. **Environment-Variablen lesen** mit `os.getenv()`
6. **Keine Default-Werte für Passwörter** (sollte explizit gesetzt sein)
7. **Passwörter niemals loggen** oder in Fehlermeldungen anzeigen
8. **README.md dokumentieren** wie `.env` erstellt wird

### Was NICHT zu tun ist:

1. **Passwörter im Code hardcodieren**
2. **`.env` Dateien committen**
3. **Passwörter in Logs ausgeben**
4. **Passwörter in Fehlermeldungen**
5. **Passwörter in Kommentaren**
6. **`.env` Dateien teilen** (jeder hat seine eigene)


In [ ]:
# Beispiel: .gitignore Eintrag prüfen/erstellen

gitignore_content = """# Environment-Variablen
.env
.env.local
.env.*.local

# Python
__pycache__/
*.pyc
*.pyo
*.pyd
.Python

# Jupyter
.ipynb_checkpoints

# Datenbanken (Beispiel)
*.db
*.sqlite
"""

print("Beispiel .gitignore Einträge:")
print(gitignore_content)
print("\nWICHTIG: Stelle sicher, dass .env in .gitignore ist!")


## 7 Zusammenfassung

### Wichtige Konzepte

1. **Environment-Variablen**: Sensible Daten außerhalb des Codes
2. **`.env` Dateien**: Lokale Konfiguration für Entwicklung
3. **`python-dotenv`**: Lädt `.env` Dateien automatisch
4. **`.gitignore`**: Verhindert, dass `.env` ins Repository kommt
5. **Best Practices**: Passwörter niemals hardcodieren oder loggen

### Praktische Anwendung

- **Datenbankverbindungen**: Host, Port, User, Passwort aus `.env`
- **API-Keys**: Externe Service-Keys aus Environment-Variablen
- **Umgebungs-Konfiguration**: Development vs. Production
- **Multi-Prozess-Anwendungen**: Sichere Konfiguration zwischen Prozessen

### Nächste Schritte

- Erstelle deine eigene `.env` Datei basierend auf `.env.example`
- Stelle sicher, dass `.env` in `.gitignore` ist
- Verwende `DatabaseConfig` Klasse als Vorlage für deine Projekte
- Dokumentiere in README.md, wie `.env` erstellt wird
